In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@databricksetesg.dfs.core.windows.net/customers")

In [0]:
df.display() 

In [0]:
df = df.drop("_rescued_data")

In [0]:
df.display()

In [0]:
df = df.withColumn("domain", split(col("email"), '@')[1])
df.display()

In [0]:
df.groupby("domain").agg(count("*").alias("total_customers")).sort("total_customers",ascending=False).display()

In [0]:
df_gmail = df.filter(col("domain")=="gmail.com")
df_gmail.display()
time.sleep(5)

df_yahoo = df.filter(col("domain")=="yahoo.com")
df_yahoo.display()
time.sleep(5)

df_hotmail = df.filter(col("domain")=="hotmail.com")
df_hotmail.display()
time.sleep(5)

In [0]:
df = df.withColumn("full_name", concat(col("first_name"),lit(' '),col("last_name")))
df.display()
df = df.drop('last_name','first_name')
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@databricksetesg.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE SCHEMA databricks_cat.silver

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cat.silver.customers_silver
USING DELTA
LOCATION "abfss://silver@databricksetesg.dfs.core.windows.net/customers"

In [0]:
%sql
SELECT * FROM databricks_cat.silver.customers_silver